# Daten der Verkehrszählung zum motorisierten Individualverkehr (MIV)

Dieses Notebook zeigt einige Beispiele, wie mit den Daten aus dem Datenatz [Daten der Verkehrszählung zum motorisierten Individualverkehr (Stundenwerte), seit 2012](https://data.stadt-zuerich.ch/dataset/sid_dav_verkehrszaehlung_miv_od2031) gearbeitet werden kann.

Der Datensatz wird **täglich** aktualisiert und beinhaltet jeweils die Daten bis und mit zum vorletzten Tag (z.B. der Datensatz vom 16.3.2020 beinhaltet die Daten bis und mit 14.3.2020.

## Inhaltsverzeichnis

1. [Datei mit Pandas einlesen](#Datei-mit-Pandas-einlesen)
2. [Übersicht der Zählstellen und deren Position (Karte)](#%C3%9Cbersicht-der-Z%C3%A4hlstellen-und-deren-Position-(Karte))
3. [Visualisierung einer Zählstelle über einen Zeitraum](#Visualisierung-einer-Zählstelle-über-einen-Zeitraum)
4. [Vergleich der Werte einer Zählstelle zwischen mehreren Wochen](#Vergleich-der-Werte-einer-Zählstelle-zwischen-mehreren-Wochen)

In [ ]:
# Python packages installieren (optional, falls nicht schon vorhanden)
!pip install requests pandas geopandas folium tqdm

In [1]:
import requests
import pandas as pd
import geopandas
import folium
from shapely.geometry import Point
import os.path
from tqdm.notebook import tqdm

In [2]:
def download_file(url, filename):
    if not filename:
        local_file = os.path.join(".",url.split('/')[-1])
    else:
        local_file = filename
        
    r = requests.get(url, stream=True)
    try:
        file_size = int(r.headers['Content-Length'])
    except KeyError:
        content_range = r.headers['Content-Range']
        file_size = int(content_range.split('/')[-1])
    
    chunk = 1
    chunk_size = 1024 # 1MB
    num_bars = int(file_size / chunk_size)
    
    #t = tqdm(total=file_size, unit_scale=True, unit_divisor=1024, desc=local_file)
    t = tqdm(total=file_size, unit_scale=True, unit_divisor=1024)
    with open(local_file, 'wb') as fp:
        for data in r.iter_content(chunk_size=chunk_size):
            t.update(len(data))
            fp.write(data)
            chunk += 1

## Datei mit Pandas einlesen

2020er Datei vom OGD-Katalog herunterladen:

In [3]:
filename = 'sid_dav_verkehrszaehlung_miv_OD2031_2020.csv'
result = requests.get(
    'https://data.stadt-zuerich.ch/api/3/action/package_show?id=sid_dav_verkehrszaehlung_miv_od2031'
)
dataset = result.json()['result']
traffic_url = None
for r in dataset['resources']:
    if r['name'] == filename:
        traffic_url = r['url']
        break

assert traffic_url, "Datei sid_dav_verkehrszaehlung_miv_OD2031_2020.csv nicht gefunden."
print("Datei %s herunterladen: " % filename)
download_file(traffic_url, filename)

Datei sid_dav_verkehrszaehlung_miv_OD2031_2020.csv herunterladen: 


File in pandas laden:

In [4]:
df = pd.read_csv(filename, encoding='utf-8')
df.head()

,MSID,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,...,Kname,AnzDetektoren,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus
0,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T00:00:00,2020-03-17,247.0,Gemessen
1,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T01:00:00,2020-03-17,364.0,Gemessen
2,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T02:00:00,2020-03-17,159.0,Gemessen
3,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T03:00:00,2020-03-17,108.0,Gemessen
4,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01T04:00:00,2020-03-17,87.0,Gemessen


In [5]:
df.columns

Index(['MSID', 'MSName', 'ZSID', 'ZSName', 'Achse', 'HNr', 'Hoehe', 'EKoord',
       'NKoord', 'Richtung', 'Knummer', 'Kname', 'AnzDetektoren', 'D1ID',
       'D2ID', 'D3ID', 'D4ID', 'MessungDatZeit', 'LieferDat', 'AnzFahrzeuge',
       'AnzFahrzeugeStatus'],
      dtype='object')

## Übersicht der Zählstellen und deren Position (Karte)

In [6]:
zaehlstellen = df[['MSID', 'MSName', 'ZSID', 'ZSName', 'Achse', 'HNr', 'Hoehe', 'EKoord',
       'NKoord', 'Richtung']]
zaehlstellen = zaehlstellen.drop_duplicates().reset_index(drop=True)
zaehlstellen

,MSID,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung
0,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts
1,Z001M002,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683013.18,1243937.64,einwärts
2,Z002M001,Unbekannt,Z002,Albisstrasse (Widmerstrasse),Albisstrasse,152,Unbekannt,2682480.51,1243162.50,auswärts
3,Z002M002,Unbekannt,Z002,Albisstrasse (Widmerstrasse),Albisstrasse,152,Unbekannt,2682484.91,1243162.41,einwärts
4,Z003M001,Unbekannt,Z003,Sood-/Leimbachstrasse,Soodstrasse,Unbekannt,Leimbachstrasse,2681573.79,1242969.91,auswärts
5,Z003M002,Unbekannt,Z003,Sood-/Leimbachstrasse,Leimbachstrasse,64,Unbekannt,2681607.86,1243010.95,einwärts
6,Z004M001,Unbekannt,Z004,Birmensdorferstrasse (Triemli),Birmensdorferstrasse,Unbekannt,90m ab Knoten,2679762.52,1246863.95,auswärts
7,Z004M002,Unbekannt,Z004,Birmensdorferstrasse (Triemli),Birmensdorferstrasse,Unbekannt,vor Knoten,2679771.84,1246852.29,einwärts
8,Z005M001,Unbekannt,Z005,Albisriederstrasse (Lyrenweg),Albisriederstrasse,Unbekannt,Lyrenweg,2678874.03,1247598.55,auswärts
9,Z005M002,Unbekannt,Z005,Albisriederstrasse (Lyrenweg),Albisriederstrasse,Unbekannt,Lyrenweg,2678871.97,1247592.79,einwärts


### DataFrame in GeoDataFrame umwandeln

Als nächsten wird das pandas DataFrame in ein GeoDataFrame von geopandas umgewandelt. Dabei wird die Projektion CH1903+ / LV95 (EPSG:2056) verwendet, da die Daten im DataFrame so vorliegen.

In [12]:
geometry = [Point(xy) for xy in zip(zaehlstellen['EKoord'], zaehlstellen['NKoord'])]
gdf = geopandas.GeoDataFrame(zaehlstellen, geometry=geometry, crs='EPSG:2056')

In [13]:
gdf.crs

<Projected CRS: EPSG:2056>
Name: CH1903+ / LV95
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe - Liechtenstein and Switzerland
- bounds: (5.96, 45.82, 10.49, 47.81)
Coordinate Operation:
- name: Swiss Oblique Mercator 1995
- method: Hotine Oblique Mercator (variant B)
Datum: CH1903+
- Ellipsoid: Bessel 1841
- Prime Meridian: Greenwich

In [28]:
gdf = gdf.to_crs('EPSG:4326')
gdf

,MSID,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,geometry
0,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,POINT (8.53707 47.34106)
1,Z001M002,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683013.18,1243937.64,einwärts,POINT (8.53711 47.34107)
2,Z002M001,Unbekannt,Z002,Albisstrasse (Widmerstrasse),Albisstrasse,152,Unbekannt,2682480.51,1243162.50,auswärts,POINT (8.52992 47.33417)
3,Z002M002,Unbekannt,Z002,Albisstrasse (Widmerstrasse),Albisstrasse,152,Unbekannt,2682484.91,1243162.41,einwärts,POINT (8.52998 47.33417)
4,Z003M001,Unbekannt,Z003,Sood-/Leimbachstrasse,Soodstrasse,Unbekannt,Leimbachstrasse,2681573.79,1242969.91,auswärts,POINT (8.51789 47.33255)
5,Z003M002,Unbekannt,Z003,Sood-/Leimbachstrasse,Leimbachstrasse,64,Unbekannt,2681607.86,1243010.95,einwärts,POINT (8.51835 47.33291)
6,Z004M001,Unbekannt,Z004,Birmensdorferstrasse (Triemli),Birmensdorferstrasse,Unbekannt,90m ab Knoten,2679762.52,1246863.95,auswärts,POINT (8.49462 47.36779)
7,Z004M002,Unbekannt,Z004,Birmensdorferstrasse (Triemli),Birmensdorferstrasse,Unbekannt,vor Knoten,2679771.84,1246852.29,einwärts,POINT (8.49474 47.36768)
8,Z005M001,Unbekannt,Z005,Albisriederstrasse (Lyrenweg),Albisriederstrasse,Unbekannt,Lyrenweg,2678874.03,1247598.55,auswärts,POINT (8.48299 47.37450)
9,Z005M002,Unbekannt,Z005,Albisriederstrasse (Lyrenweg),Albisriederstrasse,Unbekannt,Lyrenweg,2678871.97,1247592.79,einwärts,POINT (8.48296 47.37445)


### Ergebnis als Karte darstellen

In [37]:
m = folium.Map(location=[47.38, 8.53], zoom_start=13, tiles=None)
folium.raster_layers.WmsTileLayer(
    url='https://www.ogd.stadt-zuerich.ch/wms/geoportal/Basiskarte_Zuerich_Raster_Grau',
    layers='Basiskarte_Zuerich_Raster_Grau',
    name='Zürich - Basiskarte',
    fmt='image/png',
    overlay=False,
    control=False,
    autoZindex=False,
).add_to(m)

# Zählstellen als Marker hinzufügen
counter = folium.FeatureGroup("Zählstellen")
for i, row in gdf.iterrows():
    geo = row['geometry']
    html = (
        f'<h3>{row["MSID"]}</h3>'
        f'<p>{row["Achse"]} (Richtung: {row["Richtung"]})</p>'
    )
    counter.add_child(folium.Marker(location=[float(geo.y), float(geo.x)], popup=html)) 
m.add_child(counter)
m

## Visualisierung einer Zählstelle über einen Zeitraum

In [ ]:
# show progression of some values over time
stampf_data.plot.line(y='Lufttemperatur')

## Vergleich der Werte einer Zählstelle zwischen mehreren Wochen